# Find Reference Scan (change data path as needed)

In [1]:
import nibabel as nib
import numpy as np
import glob


def find_best_reference_scan(affine_matrices, median_voxel_size):
    min_diff = float("inf")
    best_scan = None

    for path, affine, voxel_size in affine_matrices:
        diff = np.sum(np.abs(voxel_size - median_voxel_size))  # Compute difference
        if diff < min_diff:
            min_diff = diff
            best_scan = path

    return best_scan


# Load all scans
image_paths = glob.glob("../Hemo_Data/raw/images/*.nii.gz")

# Store voxel sizes and affine matrices
voxel_sizes = []
affine_matrices = []

for path in image_paths:
    img = nib.load(path)
    affine = img.affine
    voxel_size = np.abs(np.diag(affine)[:3])  # Extract voxel size (spacing)

    voxel_sizes.append(voxel_size)
    affine_matrices.append((path, affine, voxel_size))

# Compute the median voxel size
median_voxel_size = np.median(voxel_sizes, axis=0)

print(f"📌 Median Voxel Size (Best for Reference): {median_voxel_size}")

best_reference_scan = find_best_reference_scan(affine_matrices, median_voxel_size)
print(f"✅ Selected Reference Scan: {best_reference_scan}")

📌 Median Voxel Size (Best for Reference): [0.48828101 0.4742834  5.        ]
✅ Selected Reference Scan: ../Hemo_Data/raw/images/Hem_00167.nii.gz


# Do Registration (change data path as needed)

In [ ]:
import ants
import os
import shutil

# Load reference scan
fixed_image = ants.image_read(best_reference_scan)

# Register all other scans
for moving_path in image_paths:
    if moving_path == best_reference_scan:
        continue  # Skip the reference scan itself

    moving_image = ants.image_read(moving_path)
    assert os.path.exists(moving_path.replace("images", "labels")), (
        "Label file not found"
    )
    moving_mask = ants.image_read(moving_path.replace("images", "labels"))

    # Perform registration (affine + deformable)
    transformation = ants.registration(
        fixed=fixed_image,
        moving=moving_image,
        type_of_transform="SyNRA",  # Symmetric Normalization (Recommended)
    )

    # Apply the computed transformations
    reg_img = ants.apply_transforms(
        fixed=fixed_image,
        moving=moving_image,
        transformlist=transformation["fwdtransforms"],  # ✅ Fixed variable name
        interpolator="linear",
        verbose=False,
    )
    reg_mask = ants.apply_transforms(
        fixed=fixed_image,
        moving=moving_mask,
        transformlist=transformation["fwdtransforms"],  # ✅ Fixed variable name
        interpolator="genericLabel",
        verbose=False,
    )

    # Save registered image
    filename = moving_path.split("/")[-1]
    output_img_path = os.path.join(f"Hemo_Data/preprocessed/images/{filename}")
    output_mask_path = os.path.join(f"Hemo_Data/preprocessed/labels/{filename}")
    ants.image_write(reg_img, output_img_path)
    ants.image_write(reg_mask, output_mask_path)

    print(f"✅ Registered {moving_path} to {best_reference_scan} → {output_img_path}")
    print(f"✅ Registered {moving_path} to {best_reference_scan} → {output_mask_path}")
    
    print(f"✅ Copied {best_reference_scan} to preprocessed folder")
    shutil.copy(
        best_reference_scan,
        f"Hemo_Data/preprocessed/images/{best_reference_scan.split('/')[-1]}",
    )
    print(f"✅ Copied {best_reference_scan.replace('images', 'labels')} to preprocessed folder")
    shutil.copy(
        best_reference_scan.replace("images", "labels"),
        f"Hemo_Data/preprocessed/labels/{best_reference_scan.split('/')[-1]}",
    )